In [24]:
import pstats
from pathlib import Path

import torch

from mlops_project.data import MyDataset

In [25]:
a = torch.arange(12).reshape(2, 6)
a

tensor([[ 0,  1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10, 11]])

In [26]:
torch.chunk(a, 2, dim=-1)

(tensor([[0, 1, 2],
         [6, 7, 8]]),
 tensor([[ 3,  4,  5],
         [ 9, 10, 11]]))

In [27]:
p = pstats.Stats("profile.txt")
p.sort_stats("cumulative").print_stats(10)

Fri Jan  9 14:18:38 2026    profile.txt

         147852761 function calls (146200170 primitive calls) in 151.930 seconds

   Ordered by: cumulative time
   List reduced from 9169 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12021    0.108    0.000  104.509    0.009 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:727(__next__)
    12021    0.303    0.000  103.572    0.009 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:786(_next_data)
    12001    0.378    0.000  102.921    0.009 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\_utils\fetch.py:47(fetch)
  1200100    9.760    0.000  100.506    0.000 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torchvision\datasets\mnist.py:131(__getitem__)
  1200100    0.719    0.000   67.317    0.000 C:\Users\otto\repos\02476-mlops-project\.venv\Lib

In [28]:
p = pstats.Stats("profile_2.txt")
p.sort_stats("cumulative").print_stats(10)

Fri Jan  9 14:18:38 2026    profile_2.txt

         19458704 function calls (19006276 primitive calls) in 52.182 seconds

   Ordered by: cumulative time
   List reduced from 9149 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
    12000    0.066    0.000   13.687    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\_tensor.py:570(backward)
    12000    0.106    0.000   13.618    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\autograd\__init__.py:243(backward)
    12000    0.056    0.000   12.912    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\autograd\graph.py:832(_engine_run_backward)
    12000   12.846    0.001   12.846    0.001 {method 'run_backward' of 'torch._C._EngineBase' objects}
    12021    0.061    0.000    8.357    0.001 C:\Users\otto\repos\02476-mlops-project\.venv\Lib\site-packages\torch\utils\data\dataloader.py:727(__next__)
    120

In [29]:
import torch
import torchvision.models as models
from torch.profiler import ProfilerActivity, profile

model = models.resnet18()
inputs = torch.randn(5, 3, 224, 224)

with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
    model(inputs)

In [30]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     aten::conv2d         0.15%     101.416us        77.47%      50.768ms       2.538ms            20  
                aten::convolution         0.36%     237.385us        77.32%      50.666ms       2.533ms            20  
               aten::_convolution         0.25%     161.996us        76.96%      50.429ms       2.521ms            20  
         aten::mkldnn_convolution        76.14%      49.894ms        76.71%      50.267ms       2.513ms            20  
                 aten::batch_norm         0.11%      70.632us         9.97%       6.536ms     326.817us            20  
     aten::_batch_norm_impl_index       

In [31]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cpu_time_total", row_limit=30))

---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls                                                                      Input Shapes  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                     aten::conv2d         0.04%      27.379us        16.26%      10.654ms       2.663ms             4                             [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], []]  
                aten::convolution         0.08%      54.407us        16.22%      10.626ms       2.657ms             4                     [[5, 64, 56, 56], [64, 64, 3, 3], [], [], [], [], [], 

In [32]:
a = torch.randn(10, 20)
a.shape

torch.Size([10, 20])

In [33]:
a.argmax(dim=1)

tensor([15,  1, 19,  9, 11,  1, 12, 11,  8, 15])

In [34]:
dataset_train = MyDataset(Path("data/processed"), train=True)

In [48]:
dataset_train.target

tensor([5, 0, 4,  ..., 6, 7, 1])